In [1]:
# 1. Post중에서 자신에게 포함된 Comment들 중에 content속성(column)에 '이한영'이라는 문자열이 들어가는 경우인 Comment가 존재하는 Post목록
#    - 일부 Post가 '이한영' 이라는 내용을 포함하는 댓글을 가져야 함
# 1-1. Post중에서 태그가 없는 경우인 Post목록
# 2. Post중에서 자신에게 포함된 Comment들 중에 Post의 author와 Comment의 author가 다른 경우인 Comment가 존재하는 Post목록
#    > 댓글은 존재하나, 작성자가 작성한 댓글은 없는 경우인 Post목록
#    - Post작성자 외에 다른 사용자가 댓글을 단 경우가 존재하여야 함
#    - Django의 F Expression을 사용 <- 검색
# 3. Tag중에서 자신에게 연결된 Post의 author의 pk가 1인 경우인 Tag목록
#    - 없다면 author의 pk를 바꿔본다
# 4. Post중에서 자신의 좋아요(PostLike)가 존재하는 Post목록
#    - Field lookup의 'isnull' 항목 참조
# 5. Comment중에서 자신의 Post의 좋아요가 존재하는 Comment목록
#    - Field lookup의 'isnull' 항목 참조 (위와 같음)
# 6. Comment중에서 자신의 Post에 속하는 가장 최근 PostLike가 1주일 이내인 Comment목록
#    - from django.utils import timezone
#    - 현재시간은 timezone.now()로 가져올 수 있음

In [1]:
Post.objects.filter(
    postcomment__content__contains='국밥',
).values('pk')

<QuerySet [{'pk': 19}, {'pk': 26}]>

In [5]:
Post.objects.filter(
    tags__isnull=True,
).values_list('pk', flat=True).order_by('-pk')

<QuerySet [25, 24, 23, 22, 21, 19, 16, 15, 8, 7, 6, 5, 4, 3, 2]>

In [14]:
Post.objects.exclude(
    postcomment__isnull=True,
).exclude(
    postcomment__author=F('author'),
).values_list('pk', flat=True).order_by('-pk')

<QuerySet [26, 25, 23, 21, 19, 8, 7]>

In [21]:
# 3. Tag중에서 자신에게 연결된 Post의 author의 pk가 1인 경우인 Tag목록
#    - 없다면 author의 pk를 바꿔본다
Tag.objects.filter(
    posts__author__pk=1
).values('pk', 'posts__pk')

<QuerySet [{'pk': 11, 'posts__pk': 1}, {'pk': 12, 'posts__pk': 1}, {'pk': 8, 'posts__pk': 10}, {'pk': 9, 'posts__pk': 10}, {'pk': 14, 'posts__pk': 10}, {'pk': 15, 'posts__pk': 10}, {'pk': 8, 'posts__pk': 11}, {'pk': 9, 'posts__pk': 11}, {'pk': 14, 'posts__pk': 11}, {'pk': 16, 'posts__pk': 12}, {'pk': 17, 'posts__pk': 13}, {'pk': 9, 'posts__pk': 27}, {'pk': 31, 'posts__pk': 27}, {'pk': 32, 'posts__pk': 28}, {'pk': 33, 'posts__pk': 28}, {'pk': 34, 'posts__pk': 28}, {'pk': 35, 'posts__pk': 28}]>

In [2]:
# 4. Post중에서 자신의 좋아요(PostLike)가 존재하는 Post목록
#    - Field lookup의 'isnull' 항목 참조
Post.objects.filter(
    postlike__isnull=False
)
Post.objects.exclude(
    postlike__isnull=True
).values_list('pk', flat=True)

<QuerySet [26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 6, 5]>

In [15]:
# 5. Comment중에서 자신의 Post의 좋아요가 존재하는 Comment목록
#    - Field lookup의 'isnull' 항목 참조 (위와 같음)
PostComment.objects.filter(
    post__postlike__isnull=False,
).values_list('pk', flat=True).distinct()

<QuerySet [41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 27, 26, 25, 24, 23, 22, 20, '...(remaining elements truncated)...']>

In [42]:
# 6. Comment중에서 자신의 Post에 속하는 가장 최근 PostLike가 3일 이후인 Comment목록
#    - from django.utils import timezone
#    - 현재시간은 timezone.now()로 가져올 수 있음
from datetime import timedelta

now = timezone.now()
a_week_ago = now - timedelta(hours=2)

Post.objects.filter(postlike__created__lt=a_week_ago).values('pk', 'postlike__user')
for item in PostComment.objects.filter(
    post__postlike__created__lt=a_week_ago
).values('pk', 'post', 'post__postlike__created').order_by('-post__pk').filter(post__pk=21):
    print(item)

{'pk': 24, 'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 33, 23, 48531, tzinfo=<UTC>)}
{'pk': 24, 'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 33, 49, 384402, tzinfo=<UTC>)}
{'pk': 24, 'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 34, 45, 564838, tzinfo=<UTC>)}
{'pk': 24, 'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 37, 7, 765532, tzinfo=<UTC>)}


In [41]:
for item in PostComment.objects.filter(
    post__pk=21
).values('post', 'post__postlike__created').order_by(''):
    print(item)

{'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 33, 23, 48531, tzinfo=<UTC>)}
{'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 33, 49, 384402, tzinfo=<UTC>)}
{'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 34, 45, 564838, tzinfo=<UTC>)}
{'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 5, 37, 7, 765532, tzinfo=<UTC>)}
{'post': 21, 'post__postlike__created': datetime.datetime(2020, 1, 15, 7, 0, 11, 591802, tzinfo=<UTC>)}


In [43]:
# 7. 각각의 Post에 댓글이 몇 개 달렸는지 QuerySet.values()로 꺼내보기
#    Django annotate / aggregate
#    QuerySetAPI의 Aggregation functions의 Count()를 사용해야 함
#    -> QuerySet.count()와 다릅니다
#   Post.objects.annotate(<무언가>).values(<왼쪽에서한걸활용>)
# 8. 위 기능에 성공했다면, 댓글이 3개 이상인 Post목록만 가져오기
#   Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)
# 9. 위 기능에 성공했다면, 댓글이 3개 이상이며 댓글개수가 좋아요개수보다 많은 Post목록만 가져오기
#   Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)
post = Post.objects.last()

In [79]:
q1 = Post.objects.annotate(comment_count=Count('postcomment', distinct=True)).filter(postcomment__isnull=False).values_list('pk', 'comment_count')
q2 = Post.objects.filter(postcomment__isnull=False).annotate(comment_count=Count('postcomment')).values_list('pk', 'comment_count')

In [80]:
q1

<QuerySet [(27, 1), (26, 3), (25, 1), (24, 4), (23, 1), (21, 1), (19, 5), (15, 4), (13, 9), (12, 2), (11, 2), (8, 6), (7, 3)]>

In [58]:
Post.objects.annotate(
    comment_count=Count('postcomment'),
).filter(
    comment_count__gte=3,
).values_list('pk', flat=True)

<QuerySet [26, 24, 19, 15, 13, 8, 7]>

In [71]:
# 9. 위 기능에 성공했다면, 댓글이 3개 이상이며 댓글개수가 좋아요개수보다 많은 Post목록만 가져오기
#   Post.objects.annotate(<무언가>).filter(<왼쪽에서한걸활용>)
Post.objects.annotate(
    comment_count=Count('postcomment'),
).annotate(
    like_count=Count('postlike'),
).filter(
#     comment_count__lte=F('like_count'),
#     comment_count__gt=F('like_count'),
).values_list('pk', 'like_count')

<QuerySet [(28, 0), (27, 3), (26, 12), (25, 3), (24, 12), (23, 4), (22, 3), (21, 5), (20, 5), (19, 15), (18, 4), (17, 5), (16, 2), (15, 16), (14, 3), (13, 54), (12, 8), (11, 8), (10, 3), (8, 6), '...(remaining elements truncated)...']>